# **Informe comparativo sobre algoritmos en Java y Python**

In [1]:
# Importacion de modulos requeridos
import pandas as pd

In [ ]:
# Leer el CSV
df_python_results = pd.read_csv('../NPC_SUB_DIGRAFH_PYTHON_CODE//results.csv', sep=';', encoding='utf-8')
df_java_results = pd.read_csv('../NPC_SUB_DIGRAFH_JAVA_CODE/results.csv')

In [ ]:
# Ver las primeras filas
print(df_python_results.head(2))

print("Datos python")
print(df_python_results.head(2))

  vertices             aristas  n  k  tiempo BF  tiempo DC  tiempo BT
0  [1...5]  [(1 : 2), (1 : 4)]  2  0   0.000375   0.000147   0.000103
1  [1...4]           [(3 : 4)]  1  1   0.000229   0.000124   0.000076
